Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов в файле “horse_data.names” , загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [1]:
import pandas as pd

In [5]:
with open ("horse_data.names", 'r') as f:
    print(f.read())

1. TItle: Horse Colic database

2. Source Information
   -- Creators: Mary McLeish & Matt Cecile
	  	Department of Computer Science
		University of Guelph
		Guelph, Ontario, Canada N1G 2W1
		mdmcleish@water.waterloo.edu
   -- Donor:    Will Taylor (taylor@pluto.arc.nasa.gov)
   -- Date:     8/6/89

3. Past Usage:
   -- Unknown

4. Relevant Information:

   -- 2 data files
      -- horse-colic.data: 300 training instances
      -- horse-colic.test: 68 test instances
   -- Possible class attributes: 24 (whether lesion is surgical)
     -- others include: 23, 25, 26, and 27
   -- Many Data types: (continuous, discrete, and nominal)

5. Number of Instances: 368 (300 for training, 68 for testing)

6. Number of attributes: 28

7. Attribute Information:

  1:  surgery?
          1 = Yes, it had surgery
          2 = It was treated without surgery

  2:  Age
          1 = Adult horse
          2 = Young (< 6 months)

  3:  Hospital Number
          - numeric id
          - the case number assi

In [72]:
column_index = [0, 1, 3, 4, 5, 6, 10, 22]
column_names = ['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']

df_horse = pd.read_csv('horse_data.csv', header=None, usecols = column_index, names = column_names, na_values='?') 
df_horse.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [27]:
df_statistics = df_horse.describe()
df_statistics

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [54]:
def emission(col_name):
    q1 = df_statistics.loc['25%', col_name]
    q3 = df_statistics.loc['75%', col_name]
    iqr = q3 - q1
    df_emission = df_horse[(df_horse[col_name] > q3 + 1.5 * iqr) | (df_horse[col_name] < q1 - 1.5 * iqr)]
    
    return df_emission

rec_temp = emission('rectal temperature')
pulse = emission('pulse')
resp = emission('respiratory rate')

print (f'Количество выбросов в rectal temperature: {len(rec_temp)}')
print (f'Количество выбросов в pulse: {len(pulse)}')
print (f'Количество выбросов в respiratory rate: {len(resp)}')

Количество выбросов в rectal temperature: 14
Количество выбросов в pulse: 5
Количество выбросов в respiratory rate: 17


Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [70]:
df_horse.isna().sum()

surgery?                       1
Age                            0
rectal temperature            60
pulse                         24
respiratory rate              58
temperature of extremities    56
pain                          55
outcome                        1
dtype: int64

Опираясь на данные о пропусках в датафрейме, можно сказать, что в столбце Age отсутствуют неизвестные данные, поэтому его можно не изменять. Столбцы surgery? и outcome содержат категориальные данные, неизвестно из которых по 1 строке в каждом. Так как это небольшое количество пропусков, то их можно удалить, не искажая общую информацию. Все остальные столбцы содержат категориальные и количественные значения, они не являются временными и количество неизвестных данных достаточно большое, поэтому есть смысл заполнить пропуски значениями для каждого столбца - для категориальных (temperature of extremities и pain) модой и для количественных медианой.

In [88]:
df_horse1 = df_horse.dropna(subset= ['surgery?', 'outcome'])

In [97]:
df_horse1['rectal temperature'].fillna(df_horse1['rectal temperature'].median(), inplace=True)
df_horse1['pulse'].fillna(df_horse1['pulse'].median(), inplace=True)
df_horse1['respiratory rate'].fillna(df_horse1['respiratory rate'].median(), inplace=True)
df_horse1['temperature of extremities'].fillna(df_horse1['temperature of extremities'].mode()[0], inplace=True)
df_horse1['pain'].fillna(df_horse1['pain'].mode()[0], inplace=True)
df_horse1.isna().sum()

/var/folders/dn/zwr5h4y129gdz0vh13yd5vgr0000gp/T/ipykernel_62174/2279069775.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_horse1['rectal temperature'].fillna(df_horse1['rectal temperature'].median(), inplace=True)
/var/folders/dn/zwr5h4y129gdz0vh13yd5vgr0000gp/T/ipykernel_62174/2279069775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

surgery?                      0
Age                           0
rectal temperature            0
pulse                         0
respiratory rate              0
temperature of extremities    0
pain                          0
outcome                       0
dtype: int64